In [1]:
import warnings
warnings.filterwarnings("ignore")

from processing_layer.fold_wise_processor import FoldWiseProcessor
import processing_layer.fold_wise_processor

from processing_layer.util.fmri_sample_cleaning_transform import get_affected_samples
from processing_layer.util.fmri_cleaner import remove_samples
import processing_layer.util.target_data_utility as target_data_utility
import os
import numpy as np
import mvpa2
import nibabel
import pickle
import gzip
from mvpa2.suite import *
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

%matplotlib inline

 * Please note: warnings are printed only once, but underlying problem might occur many times *


E:\Anaconda3\lib\site-packages\scipy\_lib\decorator.py:205: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  first = inspect.getargspec(caller)[0][0]  # first arg
E:\Anaconda3\lib\site-packages\scipy\optimize\nonlin.py:1502: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, varkw, defaults = inspect.getargspec(jac.__init__)
E:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:636: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  sign = inspect.getargspec(self._stats)
E:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:692: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  shapes_args = inspect.getargspec(meth)
E:\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.geta

In [2]:
# get the location of the files
trajectory_path = "..\\data\\story\\smoothedStoryTrajectories\\smoothTrajRawValenceMean_win500.pkl"
subject_timeline_path = "..\\data\\plosone\\subject_3.mat"
bold_fname = os.path.join('..', 'data', 'qub', '4Dw.nii')
mask_fname = os.path.join('..', 'data', 'qub', 'struct', 'final_mask_w.hdr')
ds_removal_transform = get_affected_samples(trajectory_path, subject_timeline_path)
ds = remove_samples(bold_fname, mask_fname, ds_removal_transform)
print(len(ds))
#get the trajectory
with open(trajectory_path, 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    original_trajectory = u.load()
#downsample the trajectory to fit match the number of fMRI sample
resampled_trajectory = resample(original_trajectory, len(ds.samples))

1295


In [ ]:
resampled_trajectory = target_data_utility.convert_targets_to_simple_values(resampled_trajectory)

In [3]:
ds.sa['targets'] = resampled_trajectory

In [ ]:
old_ds = ds
#ds = old_ds[:400]

In [ ]:
fold_wise_processor = FoldWiseProcessor(ds, resampled_trajectory, kNN(k=3, dfx=one_minus_correlation, voting='majority'), 10, False)
#SMLR
#fold_wise_processor = FoldWiseProcessor(ds, resampled_trajectory, SMLR(), 2, False)

In [ ]:
results = fold_wise_processor.process()

In [ ]:
for value in ds.sa.targets:
    print(value)

In [ ]:
print(fold_wise_processor.results)

In [ ]:
plt.plot(ds.sa.targets)
plt.ylabel('ds Response')
plt.show()

In [ ]:
plt.plot(fold_wise_processor.results)
plt.ylabel('response Response')
plt.show()

In [ ]:
np.corrcoef(fold_wise_processor.results, ds.sa.targets)[0, 1]

In [ ]:
np.mean(fold_wise_processor.results == ds.sa.targets)

In [ ]:
#KNN Testing
k_values = [8, 9, 10]
fold_values = [15]

for fold_value in fold_values:
    for k_value in k_values:
        temp = FoldWiseProcessor(ds, resampled_trajectory, kNN(k=k_value, dfx=one_minus_correlation, voting='majority'), fold_value, False, verbose=False)
        results = temp.process()
        
        result_value = np.mean(temp.results == ds.sa.targets)
        
        print("Fold: " + str(fold_value) + " K-Value: " + str(k_value) + " Result: " + str(result_value))


In [4]:
#Regression Testing
fold_values = [20, 15]
depth_values = [2, 3, 5, 7, 10, 15, 20, 0]
skip_values = [True, False]

print("Starting all")

for skip_value in skip_values:
    for fold_value in fold_values:
        for depth_value in depth_values:
            
            if depth_value > 0:
                temp = FoldWiseProcessor(ds, resampled_trajectory, SKLLearnerAdapter(DecisionTreeRegressor(max_depth=depth_value)), fold_value, skip_value, verbose=False)
            else:
                temp = FoldWiseProcessor(ds, resampled_trajectory, SKLLearnerAdapter(DecisionTreeRegressor()), fold_value, skip_value, verbose=False)
            results = temp.process()
            
            fileName = str(skip_value)+"_"+str(fold_value)+"_"+str(depth_value)
            np.savetxt(os.path.join('..', 'data', 'test_output', 'regression_'+fileName+'.txt'), results, fmt="%s", delimiter=',')
            
            result_value = np.corrcoef(temp.results, ds.sa.targets)[0, 1]
            
            print("Fold: " + str(fold_value) + " Depth-Value: " + str(depth_value) + " Skip " + str(skip_value) + " Result: " + str(result_value))
        print("")
    print("")

Starting all
starting to process
Fold: 20 Depth-Value: 2 Skip True Result: -0.00115807619884
starting to process
Fold: 20 Depth-Value: 3 Skip True Result: -0.0623990564346
starting to process
Fold: 20 Depth-Value: 5 Skip True Result: -0.0416922437779
starting to process
Fold: 20 Depth-Value: 7 Skip True Result: -0.0939308644178
starting to process
Fold: 20 Depth-Value: 10 Skip True Result: -0.119146628049
starting to process
Fold: 20 Depth-Value: 15 Skip True Result: -0.114249973007
starting to process
Fold: 20 Depth-Value: 20 Skip True Result: -0.107426820998
starting to process
Fold: 20 Depth-Value: 0 Skip True Result: -0.120280635917

starting to process
Fold: 15 Depth-Value: 2 Skip True Result: -0.0523909877483
starting to process


KeyboardInterrupt: 

In [ ]:
array = np.array([1, 2, 3, 4, 9, 10])

In [ ]:
np.split(array, 2)

In [ ]:
def split_list(source_list, wanted_parts=1):
    
    association_transfrom = []
    
    for i in range(0, len(source_list)):
        association_transfrom.append(i)
        
    #print(association_transfrom)
    
    length = len(association_transfrom)
    ## // notates integer division
    result_transform = [ association_transfrom[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]
    
    count = 0
    
    result = []
    
    for i in range(0, len(result_transform)):
        print(result_transform[i])
        
        for value in result_transform[i]:
            result.append(i)
            #print(value)
            #source_list[value]
            
    return result

In [ ]:
split_list([1, 2, 3, 4, 9, 10, 6], 3)

In [ ]:
len(foldwiseprocessor.generate_chunk_transform_for_ds(ds, 7))

In [ ]:
len(ds)